In [1]:
from huggingface_hub import hf_hub_download
from generator import load_csm_1b, Segment
import torchaudio
import torch


In [2]:
if torch.backends.mps.is_available():
    device = "mps"
elif torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"

print(f'device = {device}')


device = cuda


In [3]:
model_path = hf_hub_download(repo_id="sesame/csm-1b", filename="ckpt.pt")
generator = load_csm_1b(model_path, device)


/home/pushkar/Code/SesameAILabs/csm/generator.py:175: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(ckpt_path)


ckpt path or config path does not exist! Downloading the model from the Hugging Face Hub...


Fetching 13 files:   0%|          | 0/13 [00:00<?, ?it/s]

/home/pushkar/Code/SesameAILabs/csm/.venv/lib/python3.12/site-packages/silentcipher/server.py:463: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.enc_c.load_state_dict(s

In [4]:
speakers = [0]
transcripts = [
    "Oh, hello! How's it going? Why don't you tell me a joke?"
]
audio_paths = [
    "/home/pushkar/Documents/Recordings/Pushkar-Utterance.mp3",
]


In [5]:

def load_audio(audio_path):
    audio_tensor, sample_rate = torchaudio.load(audio_path)
    audio_tensor = torchaudio.functional.resample(
        audio_tensor.squeeze(0), orig_freq=sample_rate, new_freq=generator.sample_rate
    )
    return audio_tensor
    

In [6]:

segments = [
    Segment(text=transcript, speaker=speaker, audio=load_audio(audio_path)[0])
    for transcript, speaker, audio_path in zip(transcripts, speakers, audio_paths)
]


In [7]:
audio = generator.generate(
    text="Hello from Sesame.",
    speaker=0,
    context=segments,
    max_audio_length_ms=12_000,
)

torchaudio.save("audio.wav", audio.unsqueeze(0).cpu(), generator.sample_rate)
